## for BERT

In [7]:
import os

data_dir = '/home/yuan/results/epitope'
project_name = 'bert'
repo_dir = os.path.dirname(os.getcwd())
results_dir = os.path.join(repo_dir, 'models', project_name)

In [10]:
# load pre-trained model and its tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "bert-base-uncased"
# binary classification: 1, 0
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = model.config.max_position_embeddings
print('max_length', tokenizer.model_max_length)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


max_length 512


In [11]:
from datasets import load_dataset

# load data
data_files = {
    'train': [
        "/home/yuan/results/epitope/epitope.txt",
        "/home/yuan/results/epitope/random.txt",
        "/home/yuan/results/epitope/other.txt",
    ],
}
ds = load_dataset('csv', delimiter='\t', column_names=['text', 'label'], data_files=data_files, split='train')

In [12]:
ds

Dataset({
    features: ['text', 'label'],
    num_rows: 4980441
})

In [13]:
import numpy as np
import pandas as pd

def update_label(example):
    example['label'] = 1 if example['label'] == 'epitope' else 0
    return example
ds = ds.map(update_label)

# add labels: one-hot table
labels = np.array(pd.get_dummies(ds['label'], dtype='int')).tolist()
labels
ds = ds.add_column(name='labels', column=labels)
print(ds.features)

{'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}


In [14]:
ds[0]

{'text': 'RPIAEYLNTQKDM', 'label': 0, 'labels': [1]}

In [15]:
from datasets import DatasetDict

# shuffle
ds = ds.shuffle(seed=42)

# split data
train_test = ds.train_test_split(test_size=.15)
train_valid = train_test['train'].train_test_split(test_size=.15)
ds = DatasetDict({
    'train': train_valid['train'],
    'valid': train_valid['test'],
    'test': train_test['test'],
})

# tokenize
def tokenize_func(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)
ds = ds.map(tokenize_func, batched=True)

Map:   0%|          | 0/3598367 [00:00<?, ? examples/s]

Map:   0%|          | 0/635007 [00:00<?, ? examples/s]

Map:   0%|          | 0/747067 [00:00<?, ? examples/s]

In [16]:
ds

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3598367
    })
    valid: Dataset({
        features: ['text', 'label', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 635007
    })
    test: Dataset({
        features: ['text', 'label', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 747067
    })
})

In [18]:
ds['train'][0]['labels']

[1]

In [20]:
ds.save_to_disk('/home/yuan/results/epitope/epitope_bert.ds')

Saving the dataset (0/23 shards):   0%|          | 0/3598367 [00:00<?, ? examples/s]

Saving the dataset (0/4 shards):   0%|          | 0/635007 [00:00<?, ? examples/s]

Saving the dataset (0/5 shards):   0%|          | 0/747067 [00:00<?, ? examples/s]